<a href="https://colab.research.google.com/github/zeno1406/xuannhi/blob/main/rainfall_predict_LSTM_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import Necessary Libraries

In [ ]:
# Data
import numpy as np
import pandas  as pd

# torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torch.utils.data as data

# Preprocessing
from sklearn.preprocessing import MinMaxScaler

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Dataset

In [ ]:
train_df = pd.read_csv('/content/drive/Shareddrives/ngthanhlong2003@gmail.com/ML/rainfallHCMdataset.csv',delimiter=',',header=15,skipinitialspace=True)
test_df = pd.read_csv('/content/drive/Shareddrives/ngthanhlong2003@gmail.com/ML/test_dataset.csv')

# Data Visualization

In [ ]:
# Convert the YEAR, MO, and DY columns to datetimes
train_df['Date'] = pd.to_datetime(train_df['YEAR'].astype(str) + '-' + train_df['MO'].astype(str) + '-' + train_df['DY'].astype(str))
train_df.insert(0, 'Date', train_df.pop('Date'))

In [ ]:
train_df.head()

,Date,YEAR,MO,DY,T2M,QV2M,PS,WS50M,WD50M,PRECTOTCORR
0,2003-01-01,2003,1,1,25.72,16.05,100.70,3.46,88.06,0.05
1,2003-01-02,2003,1,2,25.02,15.01,100.81,3.43,98.19,0.00
2,2003-01-03,2003,1,3,25.24,14.95,100.84,3.07,109.19,0.00
3,2003-01-04,2003,1,4,25.18,14.65,100.83,3.67,99.19,0.02
4,2003-01-05,2003,1,5,25.40,14.34,100.86,3.15,76.81,0.02


In [ ]:
# Convert the YEAR, MO, and DY columns to datetimes
test_df['Date'] = pd.to_datetime(test_df['YEAR'].astype(str) + '-' + test_df['MO'].astype(str) + '-' + test_df['DY'].astype(str))
test_df.insert(0, 'Date', test_df.pop('Date'))

**Define self-defined Visualization Functions to plot distribution of each features**

In [ ]:
# Box Plot / Line Plot
class plotly_graph:
    def __init__(self, data, date):
        self.data = data
        self.date = date
        self.name_lst = ['T2M', 'QV2M', 'PS', 'WD50M', 'WS50M', 'PRECTOTCORR']
        self.box_title = 'Multiple Box Plots'
        self.line_title = 'Multiple Line Plots'

#################### function to make subplots ######################
    def make_subplot(self, graphs):
        fig = make_subplots(rows=2, cols=3, subplot_titles=(self.name_lst))
        for i in range(6): fig.add_trace(graphs[i], row=i // 3 + 1, col=i % 3 + 1)
        return fig

#################### 1. Box Plot ######################
    def box_plot(self):
        graph_lsts = []
        for i, element in enumerate(self.data.transpose()):
            graph_lst = go.Box(y = element,
                               name = self.box_title,
                               boxpoints = 'outliers',
                               line = dict(width=1))
            graph_lsts.append(graph_lst)
        fig = self.make_subplot(graph_lsts)
        fig.update_layout(title=self.box_title,
                          xaxis_title='Columns',
                          yaxis_title='Values',
                          template = 'simple_white')
        fig.show()

#################### 2. Line Plot ######################
    def line_plot(self):
        line_lsts = []
        for i, element in enumerate(self.data.transpose()):
            line = go.Scatter(x = self.date,
                               y = element,
                               mode = 'lines',
                               name = self.line_title)
            line_lsts.append(line)
        fig = self.make_subplot(line_lsts)
        fig.update_layout(title=self.line_title,
                          xaxis_title='Columns',
                          yaxis_title='Values',
                          template = 'simple_white')
        fig.show()

In [ ]:
data_ = train_df[['T2M', 'QV2M', 'PS', 'WD50M', 'WS50M', 'PRECTOTCORR']].values

graph = plotly_graph(data_, train_df['Date'])

**Box Plots:**

In [ ]:
graph.box_plot()

**Line Plots**

In [ ]:
graph.line_plot()

We can see that there's some outliers in train set, columns '' and ''.

# Feature Engineering



1.   **Get New column: 'humidity pressure ratio'**
This ratio, often called the wet-bulb potential temperature, is a measure of the air's cooling potential due to evaporation. Higher values indicate less cooling potential, often associated with drier air and higher temperatures

2.   **Get New column: 'WS50M WD50M ratio'**
This ratio gives you an idea of the prevailing wind direction at 50 meters. For example, a high ratio with a southerly wind direction indicates strong southerly winds.

In [ ]:
# Create new column 'humidity_pressure_ratio' using humidity and meanpressure

def humidity_pressure_ratio(df):
    df['humidity_pressure_ratio'] = df['QV2M'] / df['PS']
    return df

def WS50M_WD50M_ratio(df):
    df['WS50M_WD50M_ratio'] = df['WS50M'] / df['WD50M']
    return df

# apply func
train_df = humidity_pressure_ratio(train_df)
test_df = humidity_pressure_ratio(test_df)
train_df = WS50M_WD50M_ratio(train_df)
test_df = WS50M_WD50M_ratio(test_df)

# apply func
tr_date_cols = train_df['Date']
te_date_cols = test_df['Date']

train_df=train_df.set_index('Date')
test_df=test_df.set_index('Date')

In [ ]:
print('Train set \n\n')
train_df.head()

Train set 




,YEAR,MO,DY,T2M,QV2M,PS,WS50M,WD50M,PRECTOTCORR,humidity_pressure_ratio,WS50M_WD50M_ratio
Date,,,,,,,,,,,
2003-01-01,2003,1,1,25.72,16.05,100.70,3.46,88.06,0.05,0.159384,0.039291
2003-01-02,2003,1,2,25.02,15.01,100.81,3.43,98.19,0.00,0.148894,0.034932
2003-01-03,2003,1,3,25.24,14.95,100.84,3.07,109.19,0.00,0.148255,0.028116
2003-01-04,2003,1,4,25.18,14.65,100.83,3.67,99.19,0.02,0.145294,0.037000
2003-01-05,2003,1,5,25.40,14.34,100.86,3.15,76.81,0.02,0.142177,0.041010


# Modeling: Preprocess


In [ ]:
# feature selection

tr_timeseries = train_df[['MO', 'DY', 'T2M', 'QV2M', 'PS', 'WS50M', 'WD50M', 'humidity_pressure_ratio', 'WS50M_WD50M_ratio', 'PRECTOTCORR']].values.astype('float32')
te_timeseries = test_df[['MO', 'DY', 'T2M', 'QV2M', 'PS', 'WS50M', 'WD50M', 'humidity_pressure_ratio', 'WS50M_WD50M_ratio', 'PRECTOTCORR']].values.astype('float32')

new = pd.concat([train_df, test_df], axis=0).reset_index().drop('Date', axis=1)
new_timeseries = new[['MO', 'DY', 'T2M', 'QV2M', 'PS', 'WS50M', 'WD50M', 'humidity_pressure_ratio', 'WS50M_WD50M_ratio', 'PRECTOTCORR']].values.astype('float32')

# scaling using MinMax
scaler = MinMaxScaler()
tr_timeseries = scaler.fit_transform(tr_timeseries)
te_timeseries = scaler.transform(te_timeseries)

In [ ]:
# lookback: 7
tr_timeseries = train_df[['MO', 'DY', 'T2M', 'QV2M', 'PS', 'WS50M', 'WD50M', 'PRECTOTCORR', 'humidity_pressure_ratio', 'WS50M_WD50M_ratio']].values.astype('float32')
te_timeseries = test_df[['MO', 'DY', 'T2M', 'QV2M', 'PS', 'WS50M', 'WD50M', 'PRECTOTCORR', 'humidity_pressure_ratio', 'WS50M_WD50M_ratio']].values.astype('float32')
def create_dataset(dataset, lookback):
    X, y = [], []
    for i in range(len(dataset)-lookback):
        feature = dataset[:,:9][i:i+lookback]
        target = dataset[:, 9][i:i+lookback]
        X.append(feature)
        y.append(target)
    return torch.tensor(X), torch.tensor(y)

lookback = 7

train, test = tr_timeseries, te_timeseries
X_train, y_train = create_dataset(train, lookback=lookback)
X_test, y_test = create_dataset(test, lookback=lookback)

# modify shape of train and test
X_train, X_test = X_train, X_test
y_train, y_test = y_train, y_test

<ipython-input-14-ce38ac36ee3a>:11: UserWarning:

Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)



In [ ]:
loader = data.DataLoader(data.TensorDataset(X_train, y_train),
                         batch_size = 8, shuffle = True)

## Modeling

**First, define Modeling Class 'LSTMModel'**

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size = 6,
                            num_layers = 2,
                            hidden_size = 128,
                            batch_first = True,
                            bidirectional= True)

        self.dropout = nn.Dropout(0.2)
        self.linear1 = nn.Linear(128*2, 64)
        self.linear2 = nn.Linear(64, 8)
        self.output_linear = nn.Linear(8, 1)


    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.dropout(x)
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.output_linear(x)
        return x

**Call module, and define optimizer and loss function**

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# call model
model = LSTMModel()

# optimizer: Adam
optimizer = optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-5)

# loss func: MSE
loss_fn = nn.MSELoss()

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

**Define Early Stopping Function**

In [ ]:
class CustomEarlyStopping:
    def __init__(self, patience=20, delta=0, verbose=False):
        self.patience = patience
        self.delta = delta
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_state = None
        self.best_y_pred = None

    def __call__(self, val_loss, model, X):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.best_state = model.state_dict()
            with torch.no_grad():
                self.best_y_pred = model(X)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}, score: {self.best_score}')

            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_state = model.state_dict()
            with torch.no_grad():
                self.best_y_pred = model(X)
            self.counter = 0

early_stopping = CustomEarlyStopping(patience=15, verbose=True)

**Train with using pre-defined functions above**

In [ ]:
best_score = None
best_weights = None
best_train_preds = None
best_test_preds = None

n_epochs = 200

for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred.squeeze(), y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()

    with torch.no_grad():
        y_pred = model(X_train)
        train_rmse = np.sqrt(loss_fn(y_pred, y_train.unsqueeze(2)))
        train_preds = y_pred.clone().detach().cpu().numpy()

        y_pred = model(X_test)
        test_rmse = np.sqrt(loss_fn(y_pred, y_test.unsqueeze(2)))
        test_preds = y_pred.clone().detach().cpu().numpy()

        # Update the learning rate scheduler and early stopping
        scheduler.step(test_rmse)

        if best_score is None or test_rmse < best_score:
            best_score = test_rmse
            best_weights = model.state_dict()
            best_train_preds = train_preds
            best_test_preds = test_preds

        early_stopping(test_rmse, model, X_test)

        # Check if early stopping criterion is met
        if early_stopping.early_stop:
            print("Early stopping")
            break

    if epoch % 10 == 0:
        print('*'*10, 'Epoch: ', epoch, '\ train RMSE: ', train_rmse, '\ test RMSE', test_rmse)

# Final: Model Evaluation

**Get Predicted Values of Train set and Test set**

In [ ]:
if best_weights is not None:
    model.load_state_dict(best_weights)

    # Use the best weights to generate predictions
    with torch.no_grad():
        y_pred_train = model(X_train).clone().detach().cpu().numpy()
        y_pred_test = model(X_test).clone().detach().cpu().numpy()

In [ ]:
with torch.no_grad():
    train_plot = np.ones_like(new_timeseries) * np.nan
    train_plot[lookback: len(train)] = y_pred_train[:,-1,:]

    test_plot = np.ones_like(new_timeseries) * np.nan
    test_plot[len(train)+lookback:len(new_timeseries)] = y_pred_test[:,-1,:]

In [ ]:
train_predictions = scaler.inverse_transform(train_plot)
test_predictions = scaler.inverse_transform(test_plot)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

**Plot Predicted Values and Original Values**

In [ ]:
# plot
plt.figure(figsize=(20,10))
plt.plot(new_timeseries[:,9], c = 'b')
plt.plot(train_predictions[:,9], c='r')
plt.plot(test_predictions[:,9], c='g')


# plt.xlim([500,1000])
# plt.ylim([100000, 7000ㅋ00])
plt.show()